## PostgreSQL and Python

In the lecture we have talked about Databases and particulartly relational databases such as [PostgreSQL](http://www.postgresql.org). The standard language to interact with relational databases is SQL, which both serves as a 
* *Data Definition Language (DDL)* - to [define](https://www.postgresql.org/docs/9.3/static/sql-createtable.html), [modify](https://www.postgresql.org/docs/9.3/static/sql-altertable.html), and [drop]((https://www.postgresql.org/docs/9.3/static/sql-droptable.html) **tables** (as the central type of objects to store data as records in such tables), their relations, [constraints](https://www.postgresql.org/docs/9.3/static/ddl-constraints.html) on the data and other objects such as views, [indexes](https://www.postgresql.org/docs/9.3/static/sql-createindex.html), [triggers](https://www.postgresql.org/docs/9.3/static/sql-createtrigger.html), etc.
* *Data Manipultation Language (DML)* - to [insert](https://www.postgresql.org/docs/9.3/static/sql-insert.html) and [update](https://www.postgresql.org/docs/9.3/static/sql-update.html) data in tables, as well as to [query](https://www.postgresql.org/docs/9.3/static/sql-select.html) data.
 
In order to execute some SQL commands, you can go directly to the terminal and run the SQL files in a text file (typical suffix `.sql`) using `psql`, as we have seen in the slides:

      $ psql < data/entry_tutorial_example.sql
      
The file [data/entry_tutorial_example.sql](data/entry_tutorial_example.sql) actually creates tables named "Produkt" and "Preis" like in the entry tutorial.
You can now continue using `psql` and execute the queries in the slides... or use SQL directly from Python3. How?

## Using the psycopg2 Module

In Python, there are [several modules](https://wiki.postgresql.org/wiki/Python) to connect to a [PostgreSQL](http://www.postgresql.org) server, particularly, [psycopg2](http://initd.org/psycopg/) as the most popular Python driver:

In [2]:
import psycopg2

First you have to define a `connection` to the database: the function `connect()` creates a new database session and returns a new `connection` instance.  The class `connection` encapsulates a database session. It allows you to e.g. create new `cursor`s using the `cursor()` method to execute database commands and queries.

You can use a cursor to execute DDL+DML statements: the class `cursor` allows interaction with the database:
* send commands to the database using methods such as `execute()` and `executemany()`,
* retrieve data from the database by iteration or using methods such as `fetchone()`, `fetchmany()`, `fetchall()`.

Note that in the end, you have to:
 * commit changes to be written into in your database persistently (if you have done updates, insertions, or deletions
 * close both the cursor and the connection

### Setup table

taken from "./data/entry_tutorial_example.sql"

In [ ]:
#Open the connection:
conn = psycopg2.connect("dbname=postgres user=postgres") as conn:

#Open a cursor to perform database operations
cur = conn.cursor()

#-- drop the table if it already exists:
#cur.execute("DROP TABLE Produkt;")

#-- create the product table:
cur.execute("CREATE TABLE Produkt (ProduktNr integer  PRIMARY KEY NOT NULL, Bezeichnung varchar(50), Preisgruppe char(2));")
#conn.commit()
#-- insert some values:
cur.execute("INSERT INTO Produkt VALUES (1, 'Notitzblock A4 kariert', 'G3');")
cur.execute("INSERT INTO Produkt VALUES (2, 'Notitzblock A5 liniert', 'G2');")
cur.execute("INSERT INTO Produkt VALUES (3, 'Notitzblock A4 liniert', 'G3');")
cur.execute("INSERT INTO Produkt VALUES (4, 'Notitzblock A6 glatt', 'G1');")
cur.execute("INSERT INTO Produkt VALUES (5, 'Kopierpapier 500 Blatt', 'G7');")
cur.execute("INSERT INTO Produkt VALUES (6, 'Notitzblock A5 glatt', 'G3');")


#-- drop the table if it already exists:
#cur.execute("DROP TABLE Preis;")

#-- create the Preis table:
cur.execute("CREATE TABLE Preis (Preisgruppe char(2) PRIMARY KEY NOT NULL, Betrag decimal);")

cur.execute("INSERT INTO Preis VALUES ( 'G1', 0.50 );")
cur.execute("INSERT INTO Preis VALUES ( 'G2', 1.50 );")
cur.execute("INSERT INTO Preis VALUES ( 'G3', 3.00 );")
cur.execute("INSERT INTO Preis VALUES ( 'G4', 3.25 );")
cur.execute("INSERT INTO Preis VALUES ( 'G5', 5.00 );")
cur.execute("INSERT INTO Preis VALUES ( 'G6', 5.50 );")

In [ ]:
#Open the connection:
conn = psycopg2.connect("dbname=postgres user=postgres") as conn:

#Open a cursor to perform database operations
cur = conn.cursor()

# Execute a command: update the price of group 'G4' to 4.00 
cur.execute("UPDATE Preis SET Betrag = 4.0 WHERE Preisgruppe = 'G4'")

# Pass data to fill a query placeholders and let Psycopg perform
# the correct conversion (no more SQL injections!)
prodnr = 7
descr = "Neues Produkt"
group = 3

cur.execute("INSERT INTO Produkt (ProduktNr, Bezeichnung,Preisgruppe) VALUES (%s, %s, 'G%s')",
      (prodnr, descr, group))

# Query the database and obtain data as Python (tuple) objects
cur.execute("SELECT * FROM Produkt;")
print(cur.fetchall())

# Make the changes to the database persistent
conn.commit()

# Close communication (both the cursor and the connection) with the database
cur.close()
conn.close()

If you try to execute the above block twice, you get an error... Why? Hint: Look at [data/entry_tutorial_example.sql](data/entry_tutorial_example.sql) to find the answer - and recall what a [PRIMARY KEY](https://www.postgresql.org/docs/9.3/static/ddl-constraints.html) in a table is! (You should remember this from BIS I, BTW).

We can also store and query json data, or a dictionatry (as json) in Postgres:
* in order to load Json, we need the [Json](http://initd.org/psycopg/docs/extras.html#psycopg2.extras.Json) adaptor in the `psycopg2.extras` module:

In [90]:
import json
from psycopg2.extras import Json

conn = psycopg2.connect("dbname=postgres user=postgres")
cur = conn.cursor()

cur.execute("DROP TABLE jsondummytable")
cur.execute("CREATE TABLE jsondummytable (data json)")


filePath='./data/alice.json'

with open(filePath, 'r') as f:
    data=json.load(f)
    cur.execute('INSERT INTO jsondummytable VALUES(%s)', [Json(data)])

# A little bit more complex query on a json object, cf. 
#  https://www.postgresql.org/docs/9.3/static/functions-json.html
# for details:
# Get the country of the first shipping address for the record where the name is 'Alice'
cur.execute("SELECT data->'shipping_addresses'->0->>'country'as name FROM jsondummytable \
             WHERE data->>'firstname' = 'Alice';")
# BTW: note here also you can do a line break within a string in the query with a \
print(cur.fetchone()[0])

conn.commit()
cur.close()
conn.close()

Austria


## A more complex use case around our City Data use case

Now that we know how to establish some basic interaction with a database, let's try to solve the following task.
We learnt how to do filtering and merging of data with Python in [Unit3](http://).

This can be actually done within a database as well:
* Store the the EUROSTAT populations table, the iso country codes table, 
  and the indicator tables in the database in the database.
* Formulate an SQL query that gets me the 2014 average populations for cities per country, ordered by the country name.

**Discuss**: What's better? Doing it directly in Python, or doing it in SQL? Discuss pros and cons!


1st alternative to look into: store dictionaries as JSON in the database:

In [ ]:
import csv

cityCodeFile="./data/cities.csv"

conn = psycopg2.connect("dbname=postgres user=postgres")
cur = conn.cursor()

#Building the cityCode to Label map
cityCodeMap={}
with open(cityCodeFile) as f:
    csvfile = csv.reader(f)
    for i,row in enumerate(csvfile):
        cityCodeMap[row[3]]= row[1]
#cityCodeMap

# cur.execute("DROP TABLE cityCodeMap")
cur.execute("CREATE TABLE cityCodeMap (data json)")
cur.execute('INSERT INTO cityCodeMap VALUES(%s)', [Json(cityCodeMap)])

cur.execute("SELECT data->'UK004D00040' FROM cityCodeMap")
print(cur.fetchone()[0])

# Can we persist the dict as JSON in  database?
conn.commit()
cur.close()
conn.close()

Works, but doesn't seem very handy... another alternative is to load the whole CSV simply into a Table in the Database... let's write some simple code that takes a CSV file and 
* generates a table from the first row (header)
* inserts all the remaining data.
That's also probably very naive...

In [150]:
import csv

cityCodeFile="./data/cities.csv"

conn = psycopg2.connect("dbname=postgres user=postgres")
cur = conn.cursor()

# We create a table from the first 7 columns of the file: 
with open(cityCodeFile) as f:
    csvfile = csv.reader(f)

    header = "CREATE TABLE cityCodeFile_raw ("+ ', '.join([col.replace('"', '') + " varchar " for col in (next(csvfile))[:7]]) + ");"

    #cur.execute(header)

    for row in csvfile:
        cur.execute("INSERT INTO cityCodeFile_raw VALUES ("+ ','.join(["'" + str(i).replace("'", '"') + "'" for i in row[:7]]) + ")")
        

# Can we persist the dict as JSON in  database?
conn.commit()
cur.close()
conn.close()

We haven't solved this example until the end for you: To solve the overall task, you could now proceed similarly with the other tables, load them (or respectively, the relevant columns) into the database and  then work with SQL queries and joins, to get the right information out.